In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [2]:
input_x_size = 20
field_size = 2

In [4]:
vector_dimension = 3
total_plan_train_steps = 1000
batch_size = 1
all_data_size = 1000
lr = 0.01

In [5]:
MODEL_SAVE_PATH = 'TFModel'
MODEL_NAME = 'FFM'

**FFM**

$$y(x) = w_0 + \sum_{i=1}^d w_ix_i + \sum_{i=1}^d \sum_{j=i+1}^d (w_{i, f_j} \cdot w_{j,f_i}) x_i x_j$$

In [7]:
def create2dim_weight(input_x_size, field_size, vector_dimension):
    weights = tf.truncated_normal([input_x_size, field_size, vector_dimension])
    tf_weights = tf.Variable(weights)
    return tf_weights

In [8]:
def create1dim_weight(input_x_size):
    weights = tf.truncated_normal([input_x_size])
    tf_weights = tf.Variable(weights)
    return tf_weights

In [9]:
def create0dim_weight(input_x_size):
    weights = tf.truncated_normal([1])
    tf_weights = tf.Variable(weights)
    return tf_weights

![](../imgs/reduce_sum.png)
``` python
x = [
    [1, 1, 1],
    [1, 1, 1]
]
tf.reduce_sum(x) => 6
tf.reduce_sum(x, 0) => [2, 2, 2]
tf.reduce_sum(x, 1) => [3, 3]
tf.reduce_sum(x, 1, keep_dims=True) => [[3], [3]]
tf.reduce_sum(x, [0, 1]) => 6
```


**tf.gather_nd**

允许在多维上进行索引
matrix中直接通过坐标取数（索引维度和tensor维度相同）

**通过索引直接取数**
``` python
indices = [[0, 0], [1, 1]]
params = [['a', 'b'], ['c', 'd']]
output = ['a', 'd']
```

**取第1行和第2行**
```python
indices = [[1], [0]]
params = [['a', 'b'], ['c', 'd']]
output = [['c', 'd'], ['a', 'b']]
```

In [14]:
def inference(input_x, input_x_field, zero_weights, one_dim_weights, third_weight):
    # 计算回归模型预测值
    second_value = tf.reduce_sum(tf.multiply(one_dim_weights, input_x, name='second_value'))
    first_second_value = tf.add(zero_weights, second_value, name='first_second_value') 
    third_value = tf.Variable(0.0, dtype=tf.float32)
    input_shape = input_x_size
    for i in range(input_shape):
        feature_index1 = i
        field_index1 = int(input_x_field[i])
        for j in range(i+1, input_shape):
            feature_index2 = j
            field_index2 = int(input_x_field[j])
            vector_left = tf.convert_to_tensor([[feature_index1, field_index2, i] for i in range(vector_dimension)])
            weight_left = tf.gather_nd(third_weight, vector_left)
            weight_left_after_cut = tf.squeeze(weight_left)  # 删除为1的维度
            
            vector_right = tf.convert_to_tensor([[feature_index2, field_index1, i] for i in range(vector_dimension)])
            weight_right = tf.gather_nd(third_weight, vector_right)
            weight_right_after_cut = tf.squeeze(weight_right)
            
            tmp_value = tf.reduce_sum(tf.multiply(weight_left_after_cut, weight_right_after_cut))
            
            indices2 = [i]
            indices3 = [j]
            
            xi = tf.squeeze(tf.gather_nd(input_x, indices2))
            xj = tf.squeeze(tf.gather_nd(input_x, indices3))
            
            product = tf.reduce_sum(tf.multiply(xi, xj))
            second_item_val = tf.multiply(tmp_value, product)
            
            tf.assign(third_value, tf.add(third_value, second_item_val))
    return tf.add(first_second_value, third_value)

In [15]:
def gen_data():
    labels = [-1,1]
    y = [np.random.choice(labels,1)[0] for _ in range(all_data_size)]
    x_field = [i // 10 for i in range(input_x_size)]
    x = np.random.randint(0, 2, size=(all_data_size,input_x_size))
    return x, y, x_field

In [ ]:
if __name__ == '__main__':
    global_step = tf.Variable(0,trainable=False)
    trainx, trainy, trainx_field = gen_data()
    # 定义输入
    input_x = tf.placeholder(tf.float32,[input_x_size ])
    input_y = tf.placeholder(tf.float32)
    
    # 正则项的系数
    lambda_w = tf.constant(0.001, name='lambda_w')
    lambda_v = tf.constant(0.001, name='lambda_v')

    zeroWeights = create0dim_weight(input_x_size)

    oneDimWeights = create1dim_weight(input_x_size)

    thirdWeight = create2dim_weight(input_x_size,  # 创建二次项的权重变量
                                    field_size,
                                    vector_dimension)  # n * f * k

    y_ = inference(input_x, trainx_field,zeroWeights,oneDimWeights,thirdWeight)

    l2_norm = tf.reduce_sum(
        tf.add(
            tf.multiply(lambda_w, tf.pow(oneDimWeights, 2)),
            tf.reduce_sum(tf.multiply(lambda_v, tf.pow(thirdWeight, 2)),axis=[1,2]) ## 对第1和第2维进行求和处理，最终时期变成1dim
        )
    )

    loss = tf.log(1 + tf.exp(-input_y * y_)) + l2_norm

    train_step = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(total_plan_train_steps):
            for t in range(all_data_size):
                input_x_batch = trainx[t]
                input_y_batch = trainy[t]
                predict_loss,_, steps = sess.run([loss,train_step, global_step],
                                               feed_dict={input_x: input_x_batch, input_y: input_y_batch})

                print("After  {step} training   step(s)   ,   loss    on    training    batch   is  {predict_loss} "
                      .format(step=steps, predict_loss=predict_loss))

                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=steps)
                writer = tf.summary.FileWriter(os.path.join(MODEL_SAVE_PATH, MODEL_NAME), tf.get_default_graph())
                writer.close()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
After  0 training   step(s)   ,   loss    on    training    batch   is  [1.6292387] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.24313396] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.7548937] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.14759868] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.20687723] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.43344504] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.44294643] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [2.0692084] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [2.6156356] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.3449625] 
After  0 training   step(s)   ,   loss    on    tra